In [ ]:
!pip install fastapi uvicorn langgraph langchain groq sqlalchemy pydantic psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 50.9 MB/s eta 0:00:00


In [ ]:
!pip install fastapi uvicorn langgraph langchain groq sqlalchemy psycopg2-binary

In [ ]:
!pip install groq

In [ ]:
!uvicorn backend.main:app --reload --port 8000

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [7727] using WatchFiles
Process SpawnProcess-1:
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/_subprocess.py", line 80, in subprocess_started
    target(sockets=sockets)
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio_run(self.serve(sockets=sockets), loop_factory=self.config.get_loop_factory())
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^

In [ ]:
# backend/main.py
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="AI-First CRM HCP")

class ChatInput(BaseModel):
    message: str

@app.post("/interaction/chat")
def chat_interaction(data: ChatInput):
    return {
        "status": "received",
        "message": data.message
    }


In [ ]:
# backend/agent/agent.py
from langgraph.graph import StateGraph

class AgentState(dict):
    pass

def router(state):
    return "log_interaction"

graph = StateGraph(AgentState)
graph.add_node("router", router)
graph.set_entry_point("router")
agent = graph.compile()


Log Interaction

In [ ]:
# backend/tools/log_interaction.py
def log_interaction_tool(text: str):
    return {
        "hcp_name": "Dr. Sharma",
        "specialty": "Cardiology",
        "product": "Drug X",
        "summary": "Doctor showed interest",
        "sentiment": "Positive"
    }


In [ ]:
def edit_interaction_tool(interaction_id: int, updated_text: str):
    return {
        "interaction_id": interaction_id,
        "status": "updated"
    }


In [ ]:
def summarize_interactions_tool():
    return "This week: 10 interactions, 7 positive."

In [ ]:
def followup_recommendation_tool():
    return "Schedule follow-up visit in 2 weeks."

In [ ]:
def compliance_check_tool(text: str):
    return "No compliance risks detected."


In [ ]:
import os

BASE_PATH = "/content/ai_crm_hcp"
BACKEND_PATH = os.path.join(BASE_PATH, "backend")

os.makedirs(BACKEND_PATH, exist_ok=True)

open(os.path.join(BACKEND_PATH, "__init__.py"), "w").close()

print("Folders created")
print(os.listdir(BASE_PATH))
print(os.listdir(BACKEND_PATH))


Folders created
['backend']
['__init__.py']


In [ ]:
%%writefile /content/ai_crm_hcp/backend/database.py
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

DATABASE_URL = "sqlite:///./crm.db"

engine = create_engine(
    DATABASE_URL,
    connect_args={"check_same_thread": False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()



Writing /content/ai_crm_hcp/backend/database.py


In [ ]:
%%writefile /content/ai_crm_hcp/backend/models.py
from sqlalchemy import Column, Integer, String, DateTime
from datetime import datetime

from backend.database import Base

class Interaction(Base):
    __tablename__ = "interactions"

    id = Column(Integer, primary_key=True, index=True)
    hcp_name = Column(String, nullable=False)
    specialty = Column(String)
    product = Column(String)
    summary = Column(String)
    sentiment = Column(String)
    created_at = Column(DateTime, default=datetime.utcnow)


Writing /content/ai_crm_hcp/backend/models.py


In [ ]:
import sys
sys.path.insert(0, "/content/ai_crm_hcp")

print(sys.path[0])


/content/ai_crm_hcp


In [ ]:
from backend.database import Base, engine
from backend.models import Interaction

print("✅ Import successful")


✅ Import successful


In [ ]:
from backend.database import engine
from backend.models import Base

Base.metadata.create_all(bind=engine)

print("✅ Database tables created")


✅ Database tables created


backend/llm/groq_client.py

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [ ]:
import sys
sys.path.insert(0, "/content/ai_crm_hcp")


Groq client must read from environment variables.
Uses Groq
Uses llama-3.3-70b-versatile model

In [ ]:
import os
from groq import Groq

def get_groq_client():
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY not set")
    return Groq(api_key=api_key)

def call_llm(prompt: str) -> str:
    client = get_groq_client()

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  #  SUPPORTED MODEL
        messages=[
            {
                "role": "system",
                "content": "You are a healthcare CRM assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3
    )

    return response.choices[0].message.content


In [ ]:
%%writefile /content/ai_crm_hcp/backend/llm/groq_client.py
import os
from groq import Groq

def get_groq_client():
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY not set")
    return Groq(api_key=api_key)

def call_llm(prompt: str) -> str:
    client = get_groq_client()

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {
                "role": "system",
                "content": "You are a healthcare CRM assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3
    )

    return response.choices[0].message.content


Writing /content/ai_crm_hcp/backend/llm/groq_client.py


In [ ]:
import os

BASE = "/content/ai_crm_hcp/backend/llm"
os.makedirs(BASE, exist_ok=True)

open(os.path.join(BASE, "__init__.py"), "w").close()

print("llm folder contents:", os.listdir(BASE))


llm folder contents: ['__init__.py']


In [ ]:
import sys
sys.path.insert(0, "/content/ai_crm_hcp")

print(sys.path[0])


/content/ai_crm_hcp


In [ ]:
from backend.llm.groq_client import call_llm

response = call_llm(
    "Summarize this sales interaction: Met a cardiologist and discussed a new hypertension drug."
)

print("✅ Groq response:")
print(response)



✅ Groq response:
Here's a summary of the sales interaction:

**Interaction Type:** Sales Meeting
**Healthcare Professional:** Cardiologist
**Topic:** New Hypertension Drug
**Outcome:** Discussed the features and benefits of the new hypertension drug with the cardiologist.

Next steps could include:
- Following up with the cardiologist to answer any additional questions
- Providing samples or more information about the drug
- Discussing potential prescribing habits and addressing any concerns the cardiologist may have.


IMPLEMENT LANGGRAPH AGENT

In [ ]:
import os

AGENT_PATH = "/content/ai_crm_hcp/backend/agent"
os.makedirs(AGENT_PATH, exist_ok=True)

open(os.path.join(AGENT_PATH, "__init__.py"), "w").close()

print("agent folder contents:", os.listdir(AGENT_PATH))


agent folder contents: ['__init__.py']


In [ ]:
%%writefile /content/ai_crm_hcp/backend/agent/state.py
from typing import TypedDict, Optional, Any

class AgentState(TypedDict):
    user_input: str
    intent: Optional[str]
    result: Optional[Any]


Writing /content/ai_crm_hcp/backend/agent/state.py


In [ ]:
%%writefile /content/ai_crm_hcp/backend/agent/graph.py
from langgraph.graph import StateGraph, END
from backend.agent.state import AgentState

from backend.tools.log_interaction import log_interaction_tool
from backend.tools.edit_interaction import edit_interaction_tool
from backend.tools.summarize import summarize_interactions_tool
from backend.tools.followup import followup_recommendation_tool
from backend.tools.compliance import compliance_check_tool


def detect_intent(state: AgentState) -> AgentState:
    text = state["user_input"].lower()

    if "edit" in text:
        state["intent"] = "edit"
    elif "summary" in text:
        state["intent"] = "summary"
    elif "follow" in text:
        state["intent"] = "followup"
    elif "compliance" in text or "off-label" in text:
        state["intent"] = "compliance"
    else:
        state["intent"] = "log"

    return state


def log_node(state: AgentState) -> AgentState:
    state["result"] = log_interaction_tool(state["user_input"])
    return state


def edit_node(state: AgentState) -> AgentState:
    state["result"] = edit_interaction_tool(
        interaction_id=1,
        new_summary="Updated by LangGraph"
    )
    return state


def summary_node(state: AgentState) -> AgentState:
    state["result"] = summarize_interactions_tool()
    return state


def followup_node(state: AgentState) -> AgentState:
    state["result"] = followup_recommendation_tool()
    return state


def compliance_node(state: AgentState) -> AgentState:
    state["result"] = compliance_check_tool(state["user_input"])
    return state


graph = StateGraph(AgentState)

graph.add_node("router", detect_intent)
graph.add_node("log", log_node)
graph.add_node("edit", edit_node)
graph.add_node("summary", summary_node)
graph.add_node("followup", followup_node)
graph.add_node("compliance", compliance_node)

graph.set_entry_point("router")

graph.add_conditional_edges(
    "router",
    lambda state: state["intent"],
    {
        "log": "log",
        "edit": "edit",
        "summary": "summary",
        "followup": "followup",
        "compliance": "compliance",
    }
)

graph.add_edge("log", END)
graph.add_edge("edit", END)
graph.add_edge("summary", END)
graph.add_edge("followup", END)
graph.add_edge("compliance", END)

agent = graph.compile()


Writing /content/ai_crm_hcp/backend/agent/graph.py


In [ ]:
import sys
sys.path.insert(0, "/content/ai_crm_hcp")


In [ ]:
import os

TOOLS_PATH = "/content/ai_crm_hcp/backend/tools"
os.makedirs(TOOLS_PATH, exist_ok=True)

open(os.path.join(TOOLS_PATH, "__init__.py"), "w").close()

print("tools folder contents:", os.listdir(TOOLS_PATH))


tools folder contents: ['__init__.py']


In [ ]:
import sys
if "/content/ai_crm_hcp" not in sys.path:
    sys.path.insert(0, "/content/ai_crm_hcp")

print(sys.path[0])


/content/ai_crm_hcp


In [ ]:
import importlib

import backend
import backend.tools
import backend.agent

importlib.reload(backend)
importlib.reload(backend.tools)
importlib.reload(backend.agent)

print("✅ Packages reloaded")


✅ Packages reloaded


In [ ]:
%%writefile /content/ai_crm_hcp/backend/tools/log_interaction.py
from backend.llm.groq_client import call_llm
from backend.database import SessionLocal
from backend.models import Interaction

def log_interaction_tool(user_text: str):
    summary = call_llm(user_text)

    interaction = Interaction(
        hcp_name="Dr. Sharma",
        specialty="Cardiology",
        product="Hypertension Drug",
        summary=summary,
        sentiment="Positive"
    )

    db = SessionLocal()
    db.add(interaction)
    db.commit()
    db.refresh(interaction)
    db.close()

    return {
        "status": "logged",
        "interaction_id": interaction.id
    }


Writing /content/ai_crm_hcp/backend/tools/log_interaction.py


In [ ]:
%%writefile /content/ai_crm_hcp/backend/tools/edit_interaction.py
from backend.database import SessionLocal
from backend.models import Interaction

def edit_interaction_tool(interaction_id: int, new_summary: str):
    db = SessionLocal()
    interaction = db.query(Interaction).filter(
        Interaction.id == interaction_id
    ).first()

    if not interaction:
        return {"error": "Interaction not found"}

    interaction.summary = new_summary
    db.commit()
    db.close()

    return {
        "status": "updated",
        "interaction_id": interaction_id
    }


Writing /content/ai_crm_hcp/backend/tools/edit_interaction.py


In [ ]:
%%writefile /content/ai_crm_hcp/backend/tools/summarize.py
from backend.database import SessionLocal
from backend.models import Interaction

def summarize_interactions_tool():
    db = SessionLocal()
    count = db.query(Interaction).count()
    db.close()
    return f"Total interactions logged: {count}"


Writing /content/ai_crm_hcp/backend/tools/summarize.py


In [ ]:
%%writefile /content/ai_crm_hcp/backend/tools/followup.py
def followup_recommendation_tool():
    return "Recommended follow-up in 2 weeks with product samples."


Writing /content/ai_crm_hcp/backend/tools/followup.py


In [ ]:
import sys, importlib

sys.path.insert(0, "/content/ai_crm_hcp")

import backend.tools
importlib.reload(backend.tools)

print("✅ tools package loaded")


✅ tools package loaded


In [ ]:
import os

TOOLS_PATH = "/content/ai_crm_hcp/backend/tools"
os.makedirs(TOOLS_PATH, exist_ok=True)

# ensure __init__.py exists
open(os.path.join(TOOLS_PATH, "__init__.py"), "a").close()

with open(os.path.join(TOOLS_PATH, "compliance.py"), "w") as f:
    f.write(
        'def compliance_check_tool(text: str):\n'
        '    if "off-label" in text.lower():\n'
        '        return "⚠️ Potential compliance issue detected"\n'
        '    return "No compliance issues detected"\n'
    )

print("✅ compliance.py created")


✅ compliance.py created


In [ ]:
from backend.agent.graph import agent

result = agent.invoke({
    "user_input": "Met Dr. Sharma, cardiologist, discussed hypertension drug. He was interested."
})

print(result)



{'user_input': 'Met Dr. Sharma, cardiologist, discussed hypertension drug. He was interested.', 'intent': 'log', 'result': {'status': 'logged', 'interaction_id': 1}}


In [ ]:
agent.invoke({"user_input": "Give me a summary"})


{'user_input': 'Give me a summary',
 'intent': 'summary',
 'result': 'Total interactions logged: 1'}

In [ ]:
agent.invoke({"user_input": "What follow-up should I do?"})


{'user_input': 'What follow-up should I do?',
 'intent': 'followup',
 'result': 'Recommended follow-up in 2 weeks with product samples.'}

FAST API

In [ ]:
%%writefile /content/ai_crm_hcp/backend/main.py
from fastapi import FastAPI
from pydantic import BaseModel

from backend.agent.graph import agent

app = FastAPI(
    title="AI-First CRM – HCP Module",
    description="Log HCP interactions using LangGraph + LLM",
    version="1.0.0"
)


# -------------------------
# Request Schemas
# -------------------------
class InteractionRequest(BaseModel):
    user_input: str


# -------------------------
# Health Check
# -------------------------
@app.get("/")
def health_check():
    return {"status": "API is running"}


# -------------------------
# Core Endpoint (LangGraph)
# -------------------------
@app.post("/interaction")
def handle_interaction(request: InteractionRequest):
    """
    This endpoint sends user input to the LangGraph agent
    and returns the agent's structured response.
    """
    result = agent.invoke({
        "user_input": request.user_input
    })
    return result


Writing /content/ai_crm_hcp/backend/main.py



“Since I was working in a notebook environment, I validated my FastAPI endpoints using FastAPI’s TestClient, which allows end-to-end API testing without running a live server.”

In [ ]:
from fastapi.testclient import TestClient
from backend.main import app

client = TestClient(app)

response = client.post(
    "/interaction",
    json={
        "user_input": "Met Dr. Sharma, cardiologist, discussed hypertension drug. He was interested."
    }
)

print(response.json())


{'user_input': 'Met Dr. Sharma, cardiologist, discussed hypertension drug. He was interested.', 'intent': 'log', 'result': {'status': 'logged', 'interaction_id': 2}}
